<a href="https://colab.research.google.com/github/deltorobarba/machinelearning/blob/master/vqe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Variational Quantum Eigensolver (VQE)**

In [ ]:
!pip install cirq --quiet
import cirq
import sympy
from cirq.contrib.svg import SVGCircuit
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt # %matplotlib inline
print(cirq.__version__)

#### <font color="blue">**Quantum Molecule Simulation using VQE: Mathematics**

> Model different molecules and how they interact together

* **Objective**: create new, more effective medicines along with researching how medications might react with diseases such as cancer.

* **Challenge**: effectively make the calculations needed for the chemical reactions that occur. 

  * One such calculation is **calculating the ground state energy of different small molecules**.

  * Running this calculation on a classical computer can take exponential time to get an exact numerical answer.


* *Exkurs: The [ground state energy](https://en.wikipedia.org/wiki/Excited_state) is the lowest possible energy state that a molecule can be in. This is important because it gives more information about the electron configuration of the molecule.*

![gg](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/Energy_levels.svg/306px-Energy_levels.svg.png)


**From chemistry and physics we know that the Schrodinger equation tells us that if $H$ is the Hamiltonian of a system, $H|ψ⟩ = E_g|ψ⟩$ where the minimum eigenvalue $E_g$ that solves this equation corresponds to the ground state energy of the molecule**.

* To find this energy, we want to **find the exact minimum eigenvalue** that corresponds to that Hermitian matrix. 

* However that **would require a perfectly working quantum computer using the QPE (Quantum Phase Estimation Algorithm)**, which is not possible as of right now.

* Rather, we can use the [variational principle](https://en.wikipedia.org/wiki/Variational_principle) to find a close, if not exact solution.






This principle relies on the fact that a Hermitian is equal to its own conjugate transpose (ie 𝐻=𝐻†), 

* and since any eigenvalue of H must be real, all eigenvalues of 𝐻 have the property of 𝜆𝑖=𝜆∗. 

* We can rewrite the Hermitian as follows, where 𝜆 is the eigenvalue associated with the eigenvector |𝜓_i⟩

> $H=\sum_{i=1}^{N} \lambda_{i}\left|\psi_{i}\right\rangle\left\langle\psi_{i}\right|$

* which we can then use to define the expectation of this Hermitian as

> $\langle H\rangle_{\psi} \equiv\langle\psi|H| \psi\rangle$

* And finally we can substitute in for $H$ and get the following

> $\begin{aligned}\langle H\rangle_{\psi}=\langle\psi|H| \psi\rangle &=\left\langle\psi\left|\left(\sum_{i=1}^{N} \lambda_{i}\left|\psi_{i}\right\rangle\left\langle\psi_{i}\right|\right)\right| \psi\right\rangle \\ &=\sum_{i=1}^{N} \lambda_{i}\left\langle\psi \mid \psi_{i}\right\rangle\left\langle\psi_{i} \mid \psi\right\rangle \\ &=\sum_{i=1}^{N} \lambda_{i}\left|\left\langle\psi_{i} \mid \psi\right\rangle\right|^{2} \end{aligned}$

* which demonstrates that **any state can be described using a sum of the eigenvalues of the Hermitian as weights**. 

* One final property of the Hermitian is what truly allows the variational principle to work is that the eigenvalues of it have to be real, non-negative numbers, such that

> $\lambda_{\min } \leq\langle H\rangle_{\psi}=\langle\psi|H| \psi\rangle$

* And this equation is what gives power to the Variational Quantum Eigensolver. 

> **Since the minimum eigenvalue can never be smaller than the expectation value that we are looking for, then by minimizing our expectation, we will get really close to the minimum eigenvalue**.

#### <font color="blue">**Quantum Molecule Simulation using VQE: Algorithm**

https://medium.com/mit-6-s089-intro-to-quantum-computing/quantum-molecule-simulation-using-vqe-d34b9c651e3d

#### <font color="blue">**Variational method (quantum mechanics)**

https://en.m.wikipedia.org/wiki/Many-body_problem

In science and especially in mathematical studies, a [variational principle](https://en.m.wikipedia.org/wiki/Variational_principle) is one that enables a problem to be solved using calculus of variations, which concerns finding functions that optimize the values of quantities that depend on those functions.

In [quantum mechanics, the variational method](https://en.m.wikipedia.org/wiki/Variational_method_(quantum_mechanics)) is one way of finding approximations to the lowest energy eigenstate or ground state, and some excited states. This allows calculating approximate wavefunctions such as molecular orbitals. The basis for this method is the variational principle.

  * The method consists of choosing a "trial wavefunction" depending on one or more parameters, and finding the values of these parameters for which the expectation value of the energy is the lowest possible. 
  
  * The wavefunction obtained by fixing the parameters to such values is then an approximation to the ground state wavefunction, and the expectation value of the energy in that state is an upper bound to the ground state energy. **The [Hartree–Fock method](https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method), [Density matrix renormalization group](https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group), and [Ritz method](https://en.m.wikipedia.org/wiki/Ritz_method) apply the variational method**.

  * This principle relies on the fact that a Hermitian is equal to its own conjugate transpose (ie 𝐻=𝐻†), and since any eigenvalue of H must be real, all eigenvalues of 𝐻 have the property of 𝜆𝑖=𝜆∗.

**Hartree–Fock Method**

https://en.m.wikipedia.org/wiki/Hartree%E2%80%93Fock_method

https://en.m.wikipedia.org/wiki/Slater_determinant

https://en.m.wikipedia.org/wiki/Permanent_(mathematics)

**Density matrix renormalization group**

https://en.m.wikipedia.org/wiki/Density_matrix_renormalization_group

**Ritz Method**

https://en.m.wikipedia.org/wiki/Ritz_method

https://en.m.wikipedia.org/wiki/Rayleigh%E2%80%93Ritz_method